In [2]:
from datasets import load_dataset_builder, load_dataset
import numpy as np
import pandas as pd
import json

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
ds_builder = load_dataset_builder("squad_v2")
dataset = load_dataset("squad_v2", split="validation")

Found cached dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


In [130]:
all_ids = list(filter(lambda i: len(dataset[i]["answers"]["text"])>0, range(len(dataset))))


In [118]:
filtered_dataset = np.array(list(filter(lambda x : len(x["answers"]["text"])>0, dataset)))
len(filtered_dataset)

5928

In [114]:
filtered_dataset[0]

{'id': '56ddde6b9a695914005b9628',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

In [131]:
np.random.seed(42)
all_ids = list(filter(lambda i: len(dataset[i]["answers"]["text"])>0, range(len(dataset))))
subsampled_ids = np.random.choice(all_ids, size=100, replace=False)

In [125]:
df = pd.DataFrame(filtered_dataset[subsampled_ids])
filtered_dataset[0]

{'id': '56ddde6b9a695914005b9628',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

In [137]:
type(pd.DataFrame(dataset[subsampled_ids]).iloc[0]["answers"])

dict

In [141]:
df = pd.DataFrame(dataset[subsampled_ids])

converted_context = []
for ix, row in df.iterrows():
    
    # deduplication of answer spans
    new_answers = {
        "answer_start": [],
        "text": []
    }
    
    for j in range(len(row["answers"]["answer_start"])):
        is_dup = False
        for k in range(len(new_answers["text"])):
            if new_answers["answer_start"][k] == row["answers"]["answer_start"][j] and new_answers["text"][k] == row["answers"]["text"][j]:
                is_dup = True
        if not is_dup:
            new_answers["answer_start"].append(row["answers"]["answer_start"][j])
            new_answers["text"].append(row["answers"]["text"][j])

    row["answers"] = new_answers
    
    
    new_context = row["context"]
    insertions = []
    for i in range(len(row["answers"]["text"])):

        answer_span = row["answers"]["text"][i]
        start_pos = row["answers"]["answer_start"][i]
        
        insertion_pos_start = start_pos        
        insertion_pos_end = start_pos + len(answer_span)
        insertion_index = int(len(insertions)/2)
        
        insertion_str_start = f"<{insertion_index}>"
        insertion_str_end = f"</{insertion_index}>"
                
        for ins in insertions:
            if ins[0]<=insertion_pos_start:
                insertion_pos_start += len(ins[1])
            if ins[0]<=insertion_pos_end:
                insertion_pos_end += len(ins[1])
        
        new_context = "".join([new_context[0:insertion_pos_start],
            insertion_str_start,
            new_context[insertion_pos_start:insertion_pos_end],
            insertion_str_end,
            new_context[insertion_pos_end:]
        ])
        
        insertions.append((insertion_pos_start, insertion_str_start))
        insertions.append((insertion_pos_end, insertion_str_end))

    converted_context.append(new_context)
    
df["context_with_markup"] = converted_context
df.to_csv("squad_with_markup.csv")

3
5
3
5
3
3
3
3
3
3
3
3
3
4
5
3
3
3
2
3
5
3
3
3
4
4
3
3
3
3
3
3
2
3
3
3
5
5
3
4
5
3
5
3
4
3
4
5
5
3
3
3
5
3
3
5
3
3
3
3
3
3
5
3
5
3
3
3
3
3
3
3
3
3
5
3
3
4
3
3
4
3
3
3
3
3
5
4
3
3
3
3
3
3
3
5
5
3
3
3


In [5]:
df = pd.read_csv("squad_translated.csv").drop(columns=["Unnamed: 10"])
r = df.iloc[0]
print(r["context"])
print(r["answers"])

In World War II, it was recognised that the Rhine would present a formidable natural obstacle to the invasion of Germany, by the Western Allies. The Rhine bridge at Arnhem, immortalized in the book, A Bridge Too Far and the film, was a central focus of the battle for Arnhem, during the failed Operation Market Garden of September 1944. The bridges at Nijmegen, over the Waal distributary of the Rhine, were also an objective of Operation Market Garden. In a separate operation, the Ludendorff Bridge, crossing the Rhine at Remagen, became famous, when U.S. forces were able to capture it intact – much to their own surprise – after the Germans failed to demolish it. This also became the subject of a film, The Bridge at Remagen. Seven Days to the River Rhine was a Warsaw Pact war plan for an invasion of Western Europe during the Cold War.
{'answer_start': [483], 'text': ['Ludendorff Bridge']}
